In [1]:
pip install semantic-text-similarity

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ------------------------------- -------- 786.4/991.5 kB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/13.6 MB 4.2 MB/s eta 0:00:04
   --- ------------------------------------ 1.3/13.6 MB 3.7 MB/s eta 0:00:04
   ------ --------------------------------- 2.1/13.6 MB 3.8 MB/s eta 0:00:04
   -------- ------------------------------- 2.9/13.6 MB 3.8 MB/s eta 0:00:03
   ---------- ----------------------------- 3.7/13.6 MB 3.6 MB/s eta 0:00:03
   ------------- -------------------------- 4.5/13.6 MB 3.7 MB/s eta 0:00:03
   ---------------- ----------------------- 5.5/13.6 MB 3.9 MB/s eta 0:00:03
   ------------------ --------------------- 6.3/13.6 MB 3.9 MB/s eta 0:00:02
   --------------------- ------------------ 7.3/13.6 MB 4.0 MB/s eta 0:00:02
   ---

In [1]:
pip install werpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from werpy import wer as werpy_score

df = pd.read_csv("eval_manual_check_dataset.csv")

wer_scores = []
for _, row in df.iterrows():
    ideal = str(row["Ideal Answer"])
    model = str(row["Model Answer"])
    score = werpy_score(ideal, model)
    wer_scores.append(score)

df["WER"] = wer_scores
df.to_csv("eval_with_wer.csv", index=False)

In [3]:
import pandas as pd
from werpy import wer as werpy_score
from sentence_transformers import SentenceTransformer, util

baseline_files = [
    "evaluate_llm_judge/eval_check_gpt41.csv",
    "evaluate_llm_judge/eval_check_gpt40mini.csv",
    "evaluate_llm_judge/eval_check_deepseekchat.csv",
    "evaluate_llm_judge/eval_check_deepseekreasoner.csv",
    "evaluate_llm_judge/eval_check_gemini.csv"
]

model = SentenceTransformer('all-MiniLM-L6-v2')

for file in baseline_files:
    df = pd.read_csv(file)
    
    df["WER"] = df.apply(lambda row: werpy_score(str(row["Ideal Answer"]), str(row["Model Answer"])), axis=1)
    
    semantic_scores = []
    for _, row in df.iterrows():
        embeddings = model.encode([str(row["Ideal Answer"]), str(row["Model Answer"])], convert_to_tensor=True)
        score = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
        semantic_scores.append(score)
    
    df["Semantic Similarity"] = semantic_scores
    
    filename_base = file.split(".csv")[0]
    output_filename = f"{filename_base}_with_wer_semantic_v1.csv"
    
    df.to_csv(output_filename, index=False)